In [1]:
import os
import gc
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression, LogisticRegression




warnings.filterwarnings(action = 'ignore')

In [4]:
def Missing_Impute_MICE(data = None):
    """
    impute the nan values of dataset with MICE 
    """
    imputer = IterativeImputer(estimator = LinearRegression(), max_iter=30, imputation_order = 'random')
    imputer.fit(data)
    data = imputer.transform(data)